In [65]:
import csv
import numpy as np
import pandas as pd
import Loader
import tensorflow as tf
import Transformer
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import MeanSquaredError
#use for dark mode
plt.rcParams["font.family"] = "serif"
plt.style.use('dark_background')

# W/O PCA DATA

In [96]:
compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

labelsMean, trainMean = Transformer.useAverageFD(labelsTrain, compoundDataTrain)
_, testMean = Transformer.useAverageFD(labelsTest, compoundDataTest)
_, valMean = Transformer.useAverageFD(labelsValidate, compoundDataValidate)

labelsMax, trainMax = Transformer.useMaxFD(labelsTrain, compoundDataTrain)
_, testMax = Transformer.useMaxFD(labelsTest, compoundDataTest)
_, valMax = Transformer.useMaxFD(labelsValidate, compoundDataValidate)

#after transformations are done assign data
dataLabels = labelsMax
trainData = trainMax
testData = testMax
valData = valMax

trainData, testData, valData = Transformer.normalizeData(trainData, testData, valData, newMean=0, newStd=1)

print(np.shape(trainData))
for i in range(len(dataLabels)):
    print(i, "\b:\t", dataLabels[i], "\t", trainData[0:3,i])

(2717, 210)
0 :	 docking_score_max 	 [0.54137366 1.19078303 1.51548771]
1 :	 fusion_score_max 	 [0.38503803 1.38435903 0.75411238]
2 :	 maxestateindex 	 [0.53662893 0.59213859 0.87885812]
3 :	 minestateindex 	 [0.41512156 0.32720124 0.25886781]
4 :	 maxabsestateindex 	 [0.53662893 0.59213859 0.87885812]
5 :	 minabsestateindex 	 [-0.54759767 -0.75333746 -0.54125143]
6 :	 qed 	 [-0.0384085  -0.98114022 -0.64268662]
7 :	 molwt 	 [0.0205034  0.59591013 0.62484715]
8 :	 heavyatommolwt 	 [-0.04414949  0.60039371  0.65886811]
9 :	 exactmolwt 	 [0.02268469 0.59813693 0.62320799]
10 :	 numvalenceelectrons 	 [0.19907627 0.67184484 0.54290795]
11 :	 numradicalelectrons 	 [0. 0. 0.]
12 :	 maxpartialcharge 	 [-0.01918824 -0.01918824 -0.01918824]
13 :	 minpartialcharge 	 [-0.48187808 -1.05673424 -0.88605919]
14 :	 maxabspartialcharge 	 [-0.01918824 -0.01918824 -0.01918824]
15 :	 minabspartialcharge 	 [ 0.14395419  0.08354775 -0.71162518]
16 :	 fpdensitymorgan1 	 [-0.50911309 -0.2265

# With PCA DATA

In [148]:

compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

labelsPCA, trainPCA, testPCA, valPCA = Transformer.applyPCA(labelsTrain,  compoundDataTrain, 
                                                            compoundDataTest, compoundDataValidate,
                                                            endDims=[1,1,9,4,6,7,3,16])

labelsMeanPCA, trainMeanPCA = Transformer.useAverageFD(labelsPCA, trainPCA)
_, testMeanPCA = Transformer.useAverageFD(labelsPCA, testPCA)
_, valMeanPCA = Transformer.useAverageFD(labelsPCA, valPCA)

labelsMaxPCA, trainMaxPCA = Transformer.useMaxFD(labelsPCA, trainPCA)
_, testMaxPCA = Transformer.useMaxFD(labelsPCA, testPCA)
_, valMaxPCA = Transformer.useMaxFD(labelsPCA, valPCA)

#after transformations are done assign data
dataLabels = labelsMaxPCA
trainData = trainMaxPCA
testData = testMaxPCA
valData = valMaxPCA

trainDataPCA, testDataPCA, valDataPCA = Transformer.normalizeData(trainData, testData, valData, newMean=0, newStd=1)

classTrain = Transformer.toBinaryClassification(activitiesTrain)
classVal = Transformer.toBinaryClassification(activitiesValidate)
classTest = Transformer.toBinaryClassification(activitiesTest)
#print(trainDataPCA)

print(np.shape(trainDataPCA))
for i in range(len(dataLabels)):
    print(i, "\b:\t", dataLabels[i], "\t", trainData[0:3,i])

bcut2d retention: [0.99364773]
	total: 99.36477273412312%
chi retention: [0.9541968]
	total: 95.41967968597893%
paoe retention: [0.31495127 0.19509321 0.1390042  0.08838179 0.07155396 0.04590061
 0.03243    0.02673268 0.02402293]
	total: 93.80706737004682%
smr retention: [0.50634726 0.24568468 0.08792319 0.07435827]
	total: 91.43133907840047%
slogp retention: [0.44620276 0.22134465 0.15753302 0.04213752 0.03159298 0.02850562]
	total: 92.7316547196898%
estate_vsa retention: [0.29224011 0.18964809 0.14361318 0.10642656 0.07394495 0.06391846
 0.05513575]
	total: 92.49271047439824%
vsa_estate retention: [0.49255304 0.32866098 0.09719245]
	total: 91.84064669216801%
fr retention: [0.32153498 0.12578177 0.09944384 0.0570485  0.05494644 0.04503917
 0.03646285 0.02896244 0.02597271 0.02289905 0.01957315 0.01726898
 0.01514304 0.01174669 0.01137907 0.00991642]
	total: 90.31191005387316%
(2717, 95)
0 :	 docking_score_max 	 [7.9 8.  8.2]
1 :	 fusion_score_max 	 [4.96869707 5.44580412 5.37892819]

# SVM Soft Margin

In [168]:
classTrain = Transformer.toBinaryClassification(activitiesTrain)
classVal = Transformer.toBinaryClassification(activitiesValidate)
classTest = Transformer.toBinaryClassification(activitiesTest)
x = -.01
maxC=10**x
clf=svm.SVC(
    C=maxC,                          # The regularization parameter
    kernel='rbf',                   # The kernel type used 
    degree=4,                       # Degree of polynomial function 
    gamma='scale',                  # The kernel coefficient
    coef0=0.0,                      # If kernel = 'poly'/'sigmoid'
    shrinking=True,                 # To use shrinking heuristic
    probability=False,              # Enable probability estimates
    tol=0.001,                      # Stopping crierion
    cache_size=200,                 # Size of kernel cache
    class_weight=None,              # The weight of each class
    verbose=False,                  # Enable verbose output
    max_iter=- 1,                   # Hard limit on iterations
    decision_function_shape='ovr',  # One-vs-rest or one-vs-one
    break_ties=False,               # How to handle breaking ties
    random_state=None               # Random state of the model
)

type(trainData)
#my_data = np.array(trainData)
print(trainData.shape)


dataPercents = [1.0,0.9,.75,.5,.25,.1]
for percent in dataPercents:
    df = pd.DataFrame(trainDataPCA)
    df['labels'] = classTrain
    #display(df,int(len(df)*.9),len(df)*.9)
    samp = int(len(df)*percent)
    df = df.sample(n = samp)
    print(len(df))
    Ytrain = df['labels']
    del df['labels']
    Xtrain = df.values
    clf.fit(Xtrain, Ytrain)
    predictions = clf.predict(Xtrain)
    val_pred = clf.predict(valDataPCA)
    print("Percent: ", percent, " Len: ", samp, "Ein Acc: ", accuracy_score(Ytrain, predictions),
         "Eval Acc: ", accuracy_score(classVal, val_pred))


(2717, 95)
2717
Percent:  1.0  Len:  2717 Ein Acc:  0.9536253220463746 Eval Acc:  0.9147058823529411
2445
Percent:  0.9  Len:  2445 Ein Acc:  0.9529652351738241 Eval Acc:  0.9205882352941176
2037
Percent:  0.75  Len:  2037 Ein Acc:  0.9479626902307314 Eval Acc:  0.9117647058823529
1358
Percent:  0.5  Len:  1358 Ein Acc:  0.9543446244477173 Eval Acc:  0.888235294117647
679
Percent:  0.25  Len:  679 Ein Acc:  0.9499263622974963 Eval Acc:  0.8852941176470588
271
Percent:  0.1  Len:  271 Ein Acc:  0.948339483394834 Eval Acc:  0.8794117647058823


# Automated Aggregation

In [117]:
def getData():

    import csv
    import numpy as np
    import pandas as pd
    import Loader
    import Transformer

    compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
    compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
    compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

    labelsPCA, trainPCA, testPCA, valPCA = Transformer.applyPCA(labelsTrain,  compoundDataTrain, 
                                                                compoundDataTest, compoundDataValidate,
                                                                endDims=[1,1,9,4,6,7,3,16])

    labelsMeanPCA, trainMeanPCA = Transformer.useAverageFD(labelsPCA, trainPCA)
    _, testMeanPCA = Transformer.useAverageFD(labelsPCA, testPCA)
    _, valMeanPCA = Transformer.useAverageFD(labelsPCA, valPCA)

    labelsMaxPCA, trainMaxPCA = Transformer.useMaxFD(labelsPCA, trainPCA)
    _, testMaxPCA = Transformer.useMaxFD(labelsPCA, testPCA)
    _, valMaxPCA = Transformer.useMaxFD(labelsPCA, valPCA)

    #after transformations are done assign data
    dataLabels = labelsMaxPCA
    trainData = trainMaxPCA
    testData = testMaxPCA
    valData = valMaxPCA

    trainDataPCA, testDataPCA, valDataPCA = Transformer.normalizeData(trainData, testData, valData, newMean=0, newStd=1)

    classTrain = Transformer.toBinaryClassification(activitiesTrain)
    classVal = Transformer.toBinaryClassification(activitiesValidate)
    classTest = Transformer.toBinaryClassification(activitiesTest)
    
    return  trainDataPCA, testDataPCA, valDataPCA, classTrain, classVal, classTest

def aggregation(percent,rounds,model,feature_num):
    import csv
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn import preprocessing
    from sklearn import model_selection
    from tensorflow import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from sklearn import svm
    from sklearn.metrics import accuracy_score
    from tensorflow.keras.losses import BinaryCrossentropy
    from tensorflow.keras.losses import MeanSquaredError
    
    trainDataPCA, testDataPCA, valDataPCA, classTrain, classVal, classTest = getData()
    
    
    for x in rounds:
        
        curr_zero = numpy.ones(feature_num)
        aggregate = int(feature_num*percent)
        curr_zero[:aggregate] = 0
        np.random.shuffle(curr_zero)


        model.compile(loss='MeanSquaredError', optimizer="adam", metrics=['accuracy'])
        history = model.fit(trainData, Transformer.toClassification(activitiesTrain), 
                        validation_data = (valData, classVal), epochs=3, batch_size=4)
        
        curr_acc = mean(history.history["val_accuracy"])
        if rounds == 0:
            best_acc = curr
            prev_zero  = curr_zero
            
        elif curr_acc > best_acc:
            best_acc = mean(history.history["val_accuracy"])
            new_features = np.equal(curr_zero,prev_zero) 
            idx = np.where(new_features==False)
    
        else:
            prev_zero = curr_zero
            
            
            
        
    


l1Reg = 0.001

model = Sequential()
model.add(Dense(200, input_dim=np.shape(trainData)[1], activation='softmax', kernel_regularizer = keras.regularizers.L2(l1Reg)))
model.add(Dense(150, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(100, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(1, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))

model.compile(loss='MeanSquaredError', optimizer="adam", metrics=['accuracy'])
history = model.fit(trainData, Transformer.toClassification(activitiesTrain), 
                    validation_data = (valData, classVal), epochs=3, batch_size=4)

# import shap

# # load your data here, e.g. X and y
# # create and fit your model here

# # load JS visualization code to notebook
# shap.initjs()

# # explain the model's predictions using SHAP
# # (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
# explainer = shap.DeepExplainer(model,trainData)
# shap_values = explainer.shap_values(trainData)
# #print(explainer.explain_row())

# # visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
# shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

# shap.summary_plot(shap_values,trainData, plot_type="bar")

Epoch 1/3


StagingError: in user code:

    C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Tim\anaconda3\envs\tens

In [162]:
A = [1,0]
B = [1,1]
C = [A == B]
G = np.equal(A,B)
print(G)
np.where(G==False)

[ True False]


(array([1], dtype=int64),)